<a href="https://colab.research.google.com/github/Shivii-Jain/Brain-Tumor-Classification/blob/main/AI_agent_basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install google-generativeai

> Google Python SDK (google.generativeai)

In [ ]:
import google.generativeai as genai
genai.configure(api_key="Enter your API Key here")

For printing model names available based on your API key

In [ ]:
for model in genai.list_models():
  print(model.name)


Giving prompt to GenAI model gemini to generate response

In [ ]:
model = genai.GenerativeModel("gemini-1.5-flash-latest")
prompt = "After years of hard work, your hard work will take you"
response = model.generate_content(prompt)
print(response.text)

After years of hard work, your hard work will take you **to many places, depending on your goals and dedication.  It could take you to success, fulfillment, a better life, or even unexpected opportunities.**



Custom settings to tweak/ improve model performance according to your requirements.

In [ ]:
response = model.generate_content(
    "Summarize the history of the internet",
    generation_config=genai.types.GenerationConfig(
        temperature=0.7,
        top_p=0.95,
        max_output_tokens=300
    )
)

print(response.text)

The internet's history is a complex tapestry woven from military research, academic collaboration, and commercial innovation.  Here's a condensed summary:

**Early Days (1960s-1970s):**  The foundation was laid with the US Department of Defense's ARPANET project, aiming to create a decentralized communication network resilient to attack.  This involved developing packet switching, a crucial technology allowing data to be broken into smaller packets for efficient transmission.  Email emerged as an early application.

**Growth and Standardization (1980s):**  ARPANET evolved, and various networks began interconnecting, leading to the nascent internet.  Key developments included the creation of TCP/IP, the standard communication protocol that governs how data travels across the internet, and the development of the Domain Name System (DNS), making it easier to access websites using human-readable names instead of numerical addresses.

**The World Wide Web (1990s):** The invention of the Wor

To get multiple answers(candidates)

In [ ]:
response = model.generate_content(
    prompt,
    generation_config=genai.types.GenerationConfig(candidate_count=3)
)

for candidate in response.candidates:
    print(candidate.content.parts[0].text)

After years of hard work, your hard work will take you **far/to great heights/to success/where you want to be/to your goals/to a rewarding place/to unexpected opportunities.**  The best option depends on the specific context.

After years of hard work, your hard work will take you **far/to great heights/to success/where you want to be/to your goals/to a rewarding place/to the fulfillment of your dreams.**

After years of hard work, your hard work will take you **to places you never imagined.**



Model with memory

Conversation history between Roles: user and model

In [ ]:
chat = model.start_chat(history=[
    {"role": "user", "parts": ["Who is the president of India?"]},
    {"role": "model", "parts": ["As of 2024, the President of India is Droupadi Murmu."]},
    {"role": "user","parts":["After years of hard work, your hard work will take you. Complete this sentece"]},
    {"role": "model", "parts": ["After years of hard work, your hard work will take you **to many places, both expected and unexpected.**"]}

])

response = chat.send_message("What is her background?")
print(response.text)


# In this case, the model will remember that “her” refers to Droupadi Murmu — because the previous messages are included in the context.

Droupadi Murmu's background is quite remarkable.  Before becoming President, she had a long and distinguished career in Indian politics.  Here are some key aspects of her background:

* **Humble Beginnings:** She comes from a tribal community in Odisha, specifically the Santhal tribe.  She faced significant challenges early in life, highlighting her resilience and determination.

* **Teacher:**  She began her career as a teacher, demonstrating a commitment to education.

* **Politics:**  She entered politics and served as a Member of the Legislative Assembly (MLA) in Odisha for several years.  She held various ministerial portfolios in the state government, including those related to animal husbandry and fisheries, commerce and transport.

* **Tribal Advocate:** Throughout her career, she has been a strong advocate for the rights and empowerment of tribal communities in India.

* **First Tribal President:**  Her election as President of India marked a historic moment, as she is the fir

In [ ]:
response = model.generate_content(
    "What is her background?",
    generation_config=genai.types.GenerationConfig(
        temperature=0.7,
        top_p=0.95,
        max_output_tokens=100
    )
)

print(response.text)  # does not work here because cell changed

Please provide me with the name or a description of the person you're asking about.  I need more information to tell you about her background.



In [ ]:
chat_history = []

response = chat.send_message("Who is the president of the USA?")
chat_history.append({"role": "user", "parts": ["Who is the president of the USA?"]})
chat_history.append({"role": "model", "parts": [response.text]})  # can save like this
print(response.text)

The president of the USA is Joe Biden.



You can use this to track or log feedback yourself

In [ ]:
feedback = input("Was this answer correct? (yes/no): ")
if feedback.lower() == "no":
    with open("gemini_feedback_log.txt", "a") as f:
        f.write(f"Prompt: {prompt}\nResponse: {response.text}\nUser Feedback: {feedback}\n\n")


Was this answer correct? (yes/no): no


In [ ]:
chat_history

[{'role': 'user', 'parts': ['Who is the president of the USA?']},
 {'role': 'model', 'parts': ['The president of the USA is Joe Biden.\n']}]

In [ ]:
from PIL import Image

image = Image.open("/content/Y3.jpg")
model = genai.GenerativeModel("gemini-1.5-flash-latest")

response = model.generate_content([image, "What’s in this image?"])
print(response.text)


That's a magnetic resonance imaging (MRI) scan of a brain showing a brain tumor.  Specifically, it appears to be a relatively small, well-circumscribed lesion in the left parietal lobe.  The bright signal intensity suggests it might be a high-grade glioma, but a definitive diagnosis requires further analysis and correlation with clinical findings and potentially biopsy results.  This is not a diagnosis and only a radiologist can interpret an MRI.



Langchain for Agents

In [ ]:
! pip install langchain langchain-google-genai google-generativeai

In [ ]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate

os.environ["GOOGLE_API_KEY"] = "Enter your api key here"

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest", temperature=0)

# Prompt
template = ("""
Answer the following questions as best as you can. You have access to the following tools:
{tools}

Use the following format:

Question: The input question you must answer
Thought: You should always think about what to do
Action: the action is to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the answer
Final Answer: the final answer to original input question

Begin!

Question: {input}
Thought: {agent_scratchpad}
"""
)

prompt = PromptTemplate(
    template=template,
    input_variables=["tools", "tool_names", "input", "agent_scratchpad"]
)


In [ ]:
!pip install -U langchain-community

In [ ]:
!pip install -U langchain-tavily



In [ ]:
from langchain.agents import load_tools, Tool
from langchain_tavily import TavilySearch  # Can also use DuckDuckGoSearch

search = TavilySearch(tavily_api_key="Enter your API key here")

search_tool = Tool(
    name="Tavily Search",
    description="A web search engine. Use this tool as a search engine for general queries.",func=search.run
)

tools = load_tools(["llm-math"],llm=llm)
tools.append(search_tool) # => llm_math + tavily search

In [ ]:
from langchain.agents import AgentExecutor, create_react_agent

#Construct reAct agent

agent = create_react_agent(llm,tools,prompt)
agent_executor = AgentExecutor(
    agent=agent,tools=tools,verbose=True,handle_parsing_errors=True
)

In [ ]:
agent_executor.invoke(
    {
        "input":"what os the current price of a Macbook Pro in USD? How much would it cost in EUR if the exchange rate is 0.85 EUR for 1 USD? "
    }
)



> Entering new AgentExecutor chain...
Thought: I need to find the current price of a Macbook Pro in USD, then convert that price to EUR using the given exchange rate.  I'll use Tavily Search to find the price and then use the Calculator to perform the currency conversion.

Action: Tavily Search
Action Input: "current price of Macbook Pro 13 inch"{'query': 'current price of Macbook Pro 13 inch', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'MacBook Pro 13-inch - Apple', 'url': 'https://www.apple.com/macbook-pro-13/index.html', 'content': "The 13-inch MacBook Pro is supercharged by M2. It's our most portable pro laptop with all-day battery life and a gorgeous Retina display.", 'score': 0.379241, 'raw_content': None}, {'title': 'Amazon.com: Macbook Pro 13', 'url': 'https://www.amazon.com/macbook-pro-13/s?k=macbook+pro+13', 'content': 'Amazon.com: macbook pro 132025 MacBook Air 13-inch Laptop with M4 chip: Built for Apple Intelligence, 13.6-inch Liquid

{'input': 'what os the current price of a Macbook Pro in USD? How much would it cost in EUR if the exchange rate is 0.85 EUR for 1 USD? ',
 'output': 'A 13-inch Macbook Pro M2 costs approximately $1064.95 USD.  At an exchange rate of 0.85 EUR per 1 USD, this would cost approximately 905.21 EUR.'}

In [ ]:
agent_executor.invoke(
    {
        "input":"what is the current price of 24 karat gold in India? Also what is the total money gained if 1 rupee is doubled for 30 days!"
    }
)



> Entering new AgentExecutor chain...
Thought: I need to use Tavily Search to find the current price of 24 karat gold in India, and then use the Calculator to determine the total money gained if 1 rupee is doubled for 30 days.

Action: Tavily Search
Action Input: "current price of 24 karat gold in India"{'query': 'current price of 24 karat gold in India', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': "24-Karat Gold Price in India - Today's 24K Gold Rate per Gram in INR", 'url': 'https://goldpricez.com/in/24k/gram', 'content': 'The current 24K gold price per gram is ₹9,191.84 Indian Rupees (INR), showing a weekly decline of ₹177.32 (-1.89%). Explore the detailed 24 Karat gold rates in India, including live updates, price comparisons, and charts for various units such as gram, ounce, kilo, tola, and more. ... Welcome to the latest 24-Karat gold prices in India', 'score': 0.95724994, 'raw_content': None}, {'title': 'India Gold Price Live - 24-hour (gol

{'input': 'what is the current price of 24 karat gold in India? Also what is the total money gained if 1 rupee is doubled for 30 days!',
 'output': 'The current price of 24 karat gold in India is approximately ₹9,790 per gram according to the Financial Express.  If 1 rupee is doubled for 30 days, the total money gained is 1,073,741,824 rupees.'}